<a href="https://colab.research.google.com/github/NACHAMMAI-SN/Smart-Wheelchair-Control-System/blob/main/SWHEEL_CHAIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
from collections import deque

class SmartWheelchair:
    def __init__(self):
        self.battery_level = 100
        self.distance_travelled = 0
        self.wheelchair_position = np.array([0.0, 0.0])
        self.obstacles = np.random.uniform(-10, 10, (5, 2))
        self.preferences_file = 'user_preferences.txt'
        self.load_user_preferences()

    def load_user_preferences(self):
        try:
            with open(self.preferences_file, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    key, value = line.strip().split(':')
                    if key.strip() == 'wheelchair_position':
                        setattr(self, key.strip(), np.array(eval(value.strip())))
                    else:
                        setattr(self, key.strip(), eval(value.strip()))
        except FileNotFoundError:
            pass

    def save_user_preferences(self):
        preferences = {
            'battery_level': self.battery_level,
            'distance_travelled': self.distance_travelled,
            'wheelchair_position': self.wheelchair_position.tolist()
        }
        with open(self.preferences_file, 'w') as file:
            for key, value in preferences.items():
                if key == 'wheelchair_position':
                    file.write(f"{key}: {value}\n")
                else:
                    file.write(f"{key}: {value}\n")

    def is_obstacle(self, position):
        return any(np.linalg.norm(position - obs) < 0.5 for obs in self.obstacles)

    def move_wheelchair(self, direction):
        movement_map = {
            'forward': np.array([0, 0.5]),
            'backward': np.array([0, -0.5]),
            'left': np.array([-0.5, 0]),
            'right': np.array([0.5, 0])
        }
        movement = movement_map.get(direction, np.array([0, 0]))

        print(f"Moving {direction} by {movement}")

        if self.battery_level <= 0:
            self.feedback_label.value = "Battery depleted. Please charge."
            return

        new_position = self.wheelchair_position + movement

        if self.is_obstacle(new_position):
            print("Obstacle detected. Finding alternative route.")
            shortest_path = self.find_shortest_path(self.wheelchair_position, new_position)
            if shortest_path:
                for next_position in shortest_path[1:]:
                    movement = np.array(next_position) - self.wheelchair_position
                    if np.abs(movement[0]) > np.abs(movement[1]):
                        move_direction = 'right' if movement[0] > 0 else 'left'
                    else:
                        move_direction = 'forward' if movement[1] > 0 else 'backward'
                    self.move_wheelchair(move_direction)
                self.feedback_label.value = f"Reached destination: ({new_position[0]}, {new_position[1]})"
                self.save_user_preferences()
            else:
                self.feedback_label.value = "No path found to navigate around the obstacle."
        else:
            self.wheelchair_position = new_position
            self.battery_level -= np.abs(movement[0]) + np.abs(movement[1])
            self.distance_travelled += np.linalg.norm(movement)
            self.feedback_label.value = f"Moving {direction}."
            self.update_gui()

    def find_shortest_path(self, start, end):
        queue = deque([(start, [])])
        visited = set()

        while queue:
            current, path = queue.popleft()
            if np.allclose(current, end):
                return path + [current]

            if tuple(current) in visited:
                continue
            visited.add(tuple(current))

            for dx, dy in [(0.5, 0), (-0.5, 0), (0, 0.5), (0, -0.5)]:
                x, y = current[0] + dx, current[1] + dy
                if not self.is_obstacle((x, y)):
                    queue.append((np.array([x, y]), path + [current]))

        return None

    def move_to_destination(self):
        if self.battery_level <= 0:
            self.feedback_label.value = "Battery depleted. Please charge."
            return

        dest_x_input = widgets.FloatText(description='X coordinate:')
        dest_y_input = widgets.FloatText(description='Y coordinate:')
        submit_button = widgets.Button(description='Submit')
        display(widgets.VBox([dest_x_input, dest_y_input, submit_button]))

        def submit_destination(b):
            dest_x = dest_x_input.value
            dest_y = dest_y_input.value
            self.update_gui(dest_x, dest_y)
            shortest_path = self.find_shortest_path(self.wheelchair_position, np.array([dest_x, dest_y]))
            if shortest_path:
                for next_position in shortest_path[1:]:
                    movement = np.array(next_position) - self.wheelchair_position
                    if np.abs(movement[0]) > np.abs(movement[1]):
                        move_direction = 'right' if movement[0] > 0 else 'left'
                    else:
                        move_direction = 'forward' if movement[1] > 0 else 'backward'
                    self.move_wheelchair(move_direction)
                self.feedback_label.value = f"Reached destination: ({dest_x}, {dest_y})"
                self.save_user_preferences()
                self.update_gui(dest_x, dest_y)
            else:
                self.feedback_label.value = "No path found to the destination."

        submit_button.on_click(submit_destination)

    def charge_battery(self, b):
        self.battery_level = 100
        self.feedback_label.value = "Wheelchair charged."
        self.update_gui()
        self.save_user_preferences()

    def reset_charge_distance(self, b):
        self.battery_level = 100
        self.distance_travelled = 0
        self.feedback_label.value = "Charge and distance reset."
        self.update_gui()
        self.save_user_preferences()

    def reset_position(self, b):
        self.wheelchair_position = np.array([0.0, 0.0])
        self.feedback_label.value = "Position reset to origin."
        self.update_gui()
        self.save_user_preferences()

    def update_gui(self, dest_x=None, dest_y=None):
        clear_output(wait=True)
        display(widgets.VBox([self.battery_label, self.distance_label, self.feedback_label, self.control_panel, self.charge_button, self.destination_button, self.reset_button, self.reset_position_button]))
        self.battery_label.value = f'Battery Level: {self.battery_level}%'
        self.distance_label.value = f'Distance Travelled: {self.distance_travelled} meters'
        plt.figure(figsize=(8, 8))
        for ob in self.obstacles:
            plt.scatter(ob[0], ob[1], marker='x', color='black')
        plt.scatter(self.wheelchair_position[0], self.wheelchair_position[1], marker='s', color='red', label='Wheelchair')
        plt.text(self.wheelchair_position[0], self.wheelchair_position[1], f"({self.wheelchair_position[0]}, {self.wheelchair_position[1]})", verticalalignment='bottom', horizontalalignment='right')
        if dest_x is not None and dest_y is not None:
            plt.scatter(dest_x, dest_y, marker='o', color='green', label='Destination')

            shortest_path = self.find_shortest_path(self.wheelchair_position, np.array([dest_x, dest_y]))
            if shortest_path:
                path_x = [point[0] for point in shortest_path]
                path_y = [point[1] for point in shortest_path]
                if any(self.is_obstacle(point) for point in shortest_path):
                    plt.plot(path_x, path_y, color='blue', label='Shortest Path (with obstacles)')
                else:
                    plt.plot(path_x, path_y, color='green', label='Shortest Path (no obstacles)')

        plt.xlim(-25, 25)
        plt.ylim(-25, 25)
        plt.title("Smart Wheelchair")
        plt.xlabel("X position")
        plt.ylabel("Y position")
        plt.legend()
        plt.grid(True)
        plt.show()

    def run(self):

        self.battery_label = widgets.Label()
        self.distance_label = widgets.Label()
        self.feedback_label = widgets.Label()
        self.forward_button = widgets.Button(description='Forward')
        self.backward_button = widgets.Button(description='Backward')
        self.left_button = widgets.Button(description='Left')
        self.right_button = widgets.Button(description='Right')
        self.charge_button = widgets.Button(description='Charge Battery')
        self.destination_button = widgets.Button(description='Move to Destination')
        self.reset_button = widgets.Button(description='Reset Charge and Distance')
        self.reset_position_button = widgets.Button(description='Reset Position')
        self.control_panel = widgets.HBox([widgets.VBox([self.forward_button, self.backward_button]), widgets.VBox([self.left_button, self.right_button])])


        self.forward_button.on_click(lambda b: self.move_wheelchair('forward'))
        self.backward_button.on_click(lambda b: self.move_wheelchair('backward'))
        self.left_button.on_click(lambda b: self.move_wheelchair('left'))
        self.right_button.on_click(lambda b: self.move_wheelchair('right'))
        self.charge_button.on_click(self.charge_battery)
        self.destination_button.on_click(lambda b: self.move_to_destination())
        self.reset_button.on_click(self.reset_charge_distance)
        self.reset_position_button.on_click(self.reset_position)


        self.update_gui()


wheelchair_app = SmartWheelchair()
wheelchair_app.run()

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving user_preferences.txt to user_preferences (3).txt


In [ ]:
# Run this cell to upload the file
from google.colab import files
uploaded = files.upload()


Saving user_preferences.txt to user_preferences (1).txt
